This notebook is for fine-tuning the chosen small summarization model to tailor it to our specific purpose (using curated dataset with pairs of (10 reviews, 1 summary))

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import evaluate

In [ ]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/review-summarizer/inferenced_summaries.csv")

In [ ]:
# splitting dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
# splitting the train set into validation set
split_train = train_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = split_train["train"]
validation_dataset = split_train["test"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GouthamVignesh/falcon-arxiv-long-summary-1B")

def tokenize_function(examples):
    return tokenizer(examples["Top 10 Reviews"], padding="max_length", truncation=True)

# tokenize each set (into batches)
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# Breakdown of dataset for training, validation, testing: 64% train, 16% validation, 20% testing

small_train_dataset = tokenized_train_dataset.shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_test_dataset.shuffle(seed=42).select(range(1000))

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("GouthamVignesh/falcon-arxiv-long-summary-1B")

In [ ]:
# use eval strategy to monitor in fine-tuning
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
# TODO: look into ROUGE for summarization metric
metric = evaluate.load("accuracy")

# evaluation metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# create trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()